#### SEGMENTAÇÃO DE CLIENTES - APRENDIZADO NÃO SUPERVISIONADO

Vamos usar o conjunto de dados <a href='https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python' target='_new'>“Mall Customer Segmentation Data”</a> disponível no Kaggle.<p>Este conjunto de dados inclui informações básicas dos clientes do shopping, como ID do cliente, idade, gênero, renda anual e pontuação de gastos</p>

1. Importação das bibliotecas

In [1]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

2. Exploração dos dados

In [2]:
df = pd.read_csv('Mall_Customers.csv')
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
# Informação sobre o conjunto de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [5]:
# Há dados faltantes?
df.isna().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

3. Pré-processamento dos dados
<br>Vamos supor que estamos interessados na idade, renda anual e pontuação de gastos

In [6]:
atributos = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
df_scaled = StandardScaler().fit_transform(df[atributos])

4. Aplicando K-Means

In [7]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(df_scaled)

KMeans(n_clusters=3, random_state=42)

5. Adicionando os rótulos do cluster ao dataframe original

In [8]:
df['kmeans_labels'] = kmeans.labels_

In [9]:
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100),kmeans_labels
0,1,Male,19,15,39,0
1,2,Male,21,15,81,0
2,3,Female,20,16,6,0
3,4,Female,23,16,77,0
4,5,Female,31,17,40,0


6. Aplicando o algoritmo DBSCAN<br>
Os parâmetros eps e min_samples têm um grande impacto nos resultados do agrupamento.<br>
<ul>
<li>eps: especifica a distância máxima entre duas amostras para que elas sejam consideradas na mesma vizinhança. Um valor pequeno significa que mais pontos serão considerados ruídos e um valor alto significa que mais pontos serão incluídos em um cluster. O valor 'ideal' varia dependendo da distribuição dos dados.</li>
<li>min_samples:  número mínimo de amostras (ou peso total) em uma vizinhança para um ponto ser considerado como um ponto central. Isso inclui o próprio ponto. Um valor maior significa que você precisa de mais exemplos para formar um cluster.</li>
</ul>

In [10]:
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(df_scaled)

DBSCAN()

6.1 Adicionando os rótulos ao dataframe original

In [11]:
df['dbscan_labels'] = dbscan.labels_

7. Avaliação do modelo
<br>Métricas para avaliar a qualidade do agrupamento:
<ul>
<li>Índice Davies-Bouldin:Quanto menor o valor, melhor. Zero é o valor ideal.</li>
<li>Índice Calinkski-Harabasz: Quanto maior o valor, melhor</li>
</ul>

7.1 Calculando as métricas para o K-Means

In [12]:
db_kmeans = davies_bouldin_score(df_scaled, df['kmeans_labels'])
ch_kmeans = calinski_harabasz_score(df_scaled, df['kmeans_labels'])

In [13]:
print(db_kmeans)
print(ch_kmeans)

1.050270139649266
101.6949470919303


Esses valores sugerem que o modelo de agrupamento K-Means tem um desempenho razoavelmente bom

7.2 Calculando as métricas para o DBSCAN

In [14]:
db_scan = davies_bouldin_score(df_scaled, df['dbscan_labels'])
ch_dbscan = calinski_harabasz_score(df_scaled, df['dbscan_labels'])
print(db_scan)
print(ch_dbscan)

1.7569458461701164
34.07053246361936


 Esses valores sugerem que o modelo de agrupamento DBSCAN pode não estar performando tão bem quanto o modelo K-Means, com base nessas métricas. No entanto, é importante lembrar que essas métricas são apenas uma parte da história. A interpretabilidade e a utilidade dos clusters para sua aplicação específica também são muito importantes.